# Chapter 4 Examples - R

In [2]:
#### Sytem may require the following: 
# sudo apt install libcurl4-openssl-dev
# sudo apt install libxml2-dev

#### Only need to run the folloing installs once
# install.packages(c("ggplot2", "ggpubr", "tidyverse", "broom", 
#                    "AICcmodavg", "ellipse", "asbio", "QuantPsyc", 
#                    "qpcR"))

## Load Libraries

In [4]:
suppressMessages(library(readxl))
suppressMessages(library(qpcR))
#suppressMessages(library(ggplot2))
#suppressMessages(library(ggpubr))
#suppressMessages(library(tidyverse))
#suppressMessages(library(broom))
#suppressMessages(library(AICcmodavg))
#suppressMessages(library(ellipse))
#suppressMessages(library(asbio))
#suppressMessages(library(QuantPsyc))

## 4.1

In [5]:
Delivery.table <- read_excel(
    '../linear_regression_5e_data_sets/Chapter 3/Examples/data-ex-3-1 (Delivery Time).xls')

Delivery.df <- data.frame(Delivery.table$`Delivery Time, y`, 
                              Delivery.table$`Number of Cases, x1`, 
                              Delivery.table$`Distance, x2 (ft)`)

colnames(Delivery.df) <- c("DeliveryTime", "NumberCases", "Distance")

head(Delivery.df)

,DeliveryTime,NumberCases,Distance
,<dbl>,<dbl>,<dbl>
1,16.68,7,560
2,11.50,3,220
3,12.03,3,340
4,14.88,4,80
5,13.75,6,150
6,18.11,7,330


In [4]:
summary(Delivery.df)

  DeliveryTime    NumberCases       Distance     
 Min.   : 8.00   Min.   : 2.00   Min.   :  36.0  
 1st Qu.:13.75   1st Qu.: 4.00   1st Qu.: 150.0  
 Median :18.11   Median : 7.00   Median : 330.0  
 Mean   :22.38   Mean   : 8.76   Mean   : 409.3  
 3rd Qu.:21.50   3rd Qu.:10.00   3rd Qu.: 605.0  
 Max.   :79.24   Max.   :30.00   Max.   :1460.0  

In [6]:
Delivery.lm <- lm(
    DeliveryTime ~ NumberCases + Distance, 
    data=Delivery.df)  

print(Delivery.lm)


Call:
lm(formula = DeliveryTime ~ NumberCases + Distance, data = Delivery.df)

Coefficients:
(Intercept)  NumberCases     Distance  
    2.34123      1.61591      0.01438  



In [7]:
summary(Delivery.lm)


Call:
lm(formula = DeliveryTime ~ NumberCases + Distance, data = Delivery.df)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.7880 -0.6629  0.4364  1.1566  7.4197 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 2.341231   1.096730   2.135 0.044170 *  
NumberCases 1.615907   0.170735   9.464 3.25e-09 ***
Distance    0.014385   0.003613   3.981 0.000631 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.259 on 22 degrees of freedom
Multiple R-squared:  0.9596,	Adjusted R-squared:  0.9559 
F-statistic: 261.2 on 2 and 22 DF,  p-value: 4.687e-16


In [8]:
anova(Delivery.lm)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
NumberCases,1,5382.4088,5382.40880,506.61936,1.112549e-16
Distance,1,168.4021,168.40213,15.85085,6.312469e-04
Residuals,22,233.7317,10.62417,NA,NA


In [10]:
Delivery.lm$residuals

1          2          3          4          5          6          7 
-5.0280843  1.1463854 -0.0497937  4.9243539 -0.4443983 -0.2895743  0.8446235 
         8          9         10         11         12         13         14 
 1.1566049  7.4197062  2.3764129  2.2374930 -0.5930409  1.0270093  1.0675359 
        15         16         17         18         19         20         21 
 0.6712018 -0.6629284  0.4363603  3.4486213  1.7931935 -5.7879699 -2.6141789 
        22         23         24         25 
-3.6865279 -4.6075679 -4.5728535 -0.2125839

In [12]:
Delivery.lm.Mres <- anova(Delivery.lm)[[3]][3]

### Various Residual Derivations (as defined by Montgomer/Peck/Vining)

In [58]:
# Residuals
Delivery.Residuals <- resid(Delivery.lm)
Delivery.Residuals_2 <- Delivery.lm$residuals

# Check equivalency
all.equal(Delivery.Residuals, Delivery.Residuals_2)

[1] TRUE

In [60]:
# Standardized Residuals
Delivery.StandardizedResiduals <- resid(Delivery.lm) / sqrt(anova(Delivery.lm)[[3]][3])


In [67]:
# Studentized Residuals
Delivery.StudentizedResiduals <- Delivery.Residuals / sqrt(Delivery.lm.Mres * (1 - Delivery.HatMatrixDiagonal))
Delivery.StudentizedResiduals_2 <- rstandard(Delivery.lm)

# Check equivaliency
all.equal(Delivery.StudentizedResiduals, Delivery.StudentizedResiduals_2)

[1] TRUE

In [78]:
# Hat Matrix Diagonal
Delivery.HatMatrixDiagonal <- diag(Delivery.X %*% solve(t(Delivery.X) %*% Delivery.X) %*% t(Delivery.X))
Delivery.HatMatrixDiagonal_2 <- unname(influence(Delivery.lm)$hat)

all.equal(Delivery.HatMatrixDiagonal, Delivery.HatMatrixDiagonal_2)

[1] TRUE

In [85]:
# PRESS Residual
Delivery.PRESSResidual <- unname(Delivery.Residuals / (1 - Delivery.HatMatrixDiagonal))
Delivery.PRESSResidual_2 <- PRESS(Delivery.lm, verbose = FALSE)$residuals

all.equal(Delivery.PRESSResidual, Delivery.PRESSResidual_2)

[1] TRUE

In [104]:
ncol(Delivery.df)

[1] 3

In [102]:
# R-student Residual
Delivery.n = nrow(Delivery.df)
Delivery.p = ncol(Delivery.df)
S_sub_i_squared = (
        (Delivery.n - Delivery.p) * Delivery.lm.Mres - Delivery.Residuals^2 / (1 - Delivery.HatMatrixDiagonal)
    ) / (
        (Delivery.n - Delivery.p - 1)
    )
Delivery.RStudentResidual <- Delivery.Residuals / sqrt(S_sub_i_squared * (1 - Delivery.HatMatrixDiagonal))

In [105]:
Delivery.n - Delivery.p

[1] 22

In [106]:
Delivery.lm.Mres

[1] 10.62417

In [107]:
Delivery.Residuals[9]

9 
7.419706

In [108]:
Delivery.HatMatrixDiagonal[9]

[1] 0.4982922

In [103]:
Delivery.RStudentResidual

1           2           3           4           5           6 
-1.69562881  0.35753764 -0.01572177  1.63916491 -0.13856493 -0.08873728 
          7           8           9          10          11          12 
 0.26464769  0.35938983  4.31078012  0.80677584  0.70993906 -0.18897451 
         13          14          15          16          17          18 
 0.31846924  0.33417725  0.20566324 -0.21782566  0.13492400  1.11933065 
         19          20          21          22          23          24 
 0.56981420 -1.99667657 -0.87308697 -1.48962473 -1.48246718 -1.54221512 
         25 
-0.06596332

In [101]:
rstudent(Delivery.lm)

1           2           3           4           5           6 
-1.69562881  0.35753764 -0.01572177  1.63916491 -0.13856493 -0.08873728 
          7           8           9          10          11          12 
 0.26464769  0.35938983  4.31078012  0.80677584  0.70993906 -0.18897451 
         13          14          15          16          17          18 
 0.31846924  0.33417725  0.20566324 -0.21782566  0.13492400  1.11933065 
         19          20          21          22          23          24 
 0.56981420 -1.99667657 -0.87308697 -1.48962473 -1.48246718 -1.54221512 
         25 
-0.06596332

In [38]:
Delivery.Residuals <- resid(Delivery.lm)
Delivery.StandardizedResiduals <- Delivery.Residuals / sqrt(Delivery.lm.Mres)
Delivery.HatMatrixDiagonal <- diag(Delivery.X %*% solve(t(Delivery.X) %*% Delivery.X) %*% t(Delivery.X))
Delivery.StudentizedResiduals <- Delivery.Residuals / sqrt(Delivery.lm.Mres * (1 - Delivery.HatMatrixDiagonal))

Delivery.StudentizedResiduals_2 <- rstandard(Delivery.lm)
Delivery.RStudentizedResiduals <- rstudent(Delivery.lm)
Delivery.pressres <- PRESS(Delivery.lm, verbose = FALSE)$residuals
Delivery.pressres_sq <- Delivery.pressres ^ 2
Delivery.pressstat <- PRESS(Delivery.lm, verbose = FALSE)$stat
Delivery.X <- as.matrix(cbind(1, Delivery.df$NumberCases, Delivery.df$Distance))


In [32]:
Delivery.Residual_df <- data.frame(
    unname(Delivery.Residuals),
    unname(Delivery.StandardizedResiduals),
    unname(Delivery.StudentizedResiduals)
)

In [33]:
head(Delivery.Residual_df)

,unname.Delivery.Residuals.,unname.Delivery.StandardizedResiduals.,unname.Delivery.StudentizedResiduals.
,<dbl>,<dbl>,<dbl>
1,-5.0280843,-1.54260631,-1.62767993
2,1.1463854,0.35170879,0.36484267
3,-0.0497937,-0.01527661,-0.01609165
4,4.9243539,1.51078203,1.57972040
5,-0.4443983,-0.13634053,-0.14176094
6,-0.2895743,-0.08884082,-0.09080847


In [17]:
colnames(Delivery.Residual_df) <- c(
    "Residuals",
    "res_hm",
    "UNKNOWN", 
    "stdres", 
    "studres", 
    "hatmatdiag", 
    "pressres", 
    "pressres_sq")

In [94]:
stdres(Delivery.lm)

1           2           3           4           5           6 
-1.62767993  0.36484267 -0.01609165  1.57972040 -0.14176094 -0.09080847 
          7           8           9          10          11          12 
 0.27042496  0.36672118  3.21376278  0.81325432  0.71807970 -0.19325733 
         13          14          15          16          17          18 
 0.32517935  0.34113547  0.21029137 -0.22270023  0.13803929  1.11295196 
         19          20          21          22          23          24 
 0.57876634 -1.87354643 -0.87784258 -1.44999541 -1.44368977 -1.49605875 
         25 
-0.06750861